# Plantilla para hacer gráficas hermosas en python

In [15]:
#____________________________________LIBRERÍAS GENERALES
import numpy as np #para operaciones
import pandas as pd #para manipular archivos en excel
import matplotlib.pyplot as plt #para graficar
from matplotlib import rc #para cambiar estilo de fuente de texto
rc('font',**{'family':'sans-serif','sans-serif':['Helvetica']}) #opción1
rc('font',**{'family':'serif','serif':['Times']}) #opción2
rc('text', usetex=True)

#____________________________________LIBRERÍA PARA CONSTANTES FÍSICAS
import astropy
from astropy import units as un
from astropy import constants as cte

#____________________________________LIBRERÍAS DE PROCESAMIENTO
from scipy import interpolate
from scipy.optimize import curve_fit
import scipy.stats as stats

## Cómo obtener los datos

In [ ]:
#______________________________________DE .xlsx
df = pd.read_excel('datoslab.xlsx')

corriente = np.array(df['Columna1'][:]) #traducidos de listas a matrices
voltaje =  df['Columna2'][:] #sin traducción

df.to_excel('datos.xlsx', index=False)

#______________________________________DE .txt
data = np.loadtxt('data_FH.txt',comments='#',delimiter=',')

#______________________________________DE GIT HUB
I = data[:,0]
V = data[:,1]

## Tratamientos de datos

In [ ]:
# Encontrar índices de valores únicos (sin las repeticiones) en corriente
indices_unicos = np.unique(corriente, return_index=True)[1]

# Filtrar corriente según los índices únicos
corriente_filtrada = corriente[indices_unicos]

### Interpolaciones, suavizados, etc

In [ ]:
#SUAVIZADO DE CURVA CON VENTANAS MÓVILES

# Crear un DataFrame de pandas para trabajar con los datos
dff = pd.DataFrame({'Datos': datos})
# Aplicar un filtro de media móvil para suavizar los datos
window_size = 5
dff['Suavizado'] = dff['Datos'].rolling(window=window_size).mean()
datos_suavizados = np.array(dff['Suavizado'])

In [ ]:
#INTERPOLACIÓN POR MÉTODO DE 

fun = interpolate.interp1d(V, I,kind='cubic')

In [ ]:
#REGRESIÓN LINEAL

# Realizar la regresión lineal
slope, intercept, r_value, p_value, std_err = stats.linregress(datosX, datosY)

# Calcular la desviación estándar de la pendiente
std_dev_slope = std_err

# Calcular R^2
r_squared = r_value ** 2

# Varianza en la ordenada (b)
var_b = covariance_R1[1, 1]

# Incertidumbre en la ordenada
uncertainty_b = np.sqrt(var_b)

# Imprimir la pendiente y la desviación estándar de la pendiente
print(f'Pendiente de la regresión: {slope:.4f}')
print(f'Desviación estándar de la pendiente: {std_dev_slope:.4f}')
print(f'Ordenada de la regresión: {intercept:.4f}')
print(f"Incertidumbre en la ordenada (b): ±{uncertainty_b:.2f}")
print(f"Coeficiente de determinación (R^2): {r_squared}")

In [ ]:
#AJUSTE DE DATOS A UNA FUNCIÓN

# Función de ajuste
def func(V, a, b):
    return (a * V) + b

# Realizar ajuste de curva para D1
params_D1, covariance_D1 = curve_fit(func, V, D1)
a_D1, b_D1 = params_D1

# Generar valores ajustados
V_fit = np.linspace(min(V), max(V), 100)
D1_fit = func(V_fit, a_D1, b_D1)

In [ ]:
#AJUSTE DE DATOS A UNA FUNCIÓN (ejemplo 2)

# Función de ajuste lineal sin ordenada
def linear_fit(x, m):
    return m * x

# Ajuste lineal sin ordenada
popt, pcov = curve_fit(linear_fit, datosX, datosY)

# Obtener la pendiente de regresión
slope = popt[0]

# Obtener la desviación estándar de la pendiente
slope_std_dev = np.sqrt(pcov[0, 0])

# Calcular el valor predicho utilizando la función ajustada
y_pred = linear_fit(datosX, *popt)

# Calcular el coeficiente de determinación (R^2)
ss_residual = np.sum((datosY - y_pred)**2)
ss_total = np.sum((datosY - np.mean(datosY))**2)
r_squared = 1 - (ss_residual / ss_total)

# Imprimir resultados
print("Pendiente de regresión:", slope)
print("Desviación estándar de la pendiente:", slope_std_dev)
print("Coeficiente de determinación (R^2):", r_squared)

## Imprimir Resultados en Pantalla

In [ ]:
#_____________________________________________________________FORMAS DE IMPRESIÓN
#Opción 1:
print("La corriente es:", corriente_filtrada, "[nA]")
# Opción 2:
print('La corriente es {} [nA]'.format(corriente_filtrada))

#_____________________________________________________________IMPRESIONES
#Para imprimir super y sub índices hay que conocer la tabla de Unicode
#Ejemplo:
print(u'H\u2082SO\u2084') # H₂SO₄

## Imprimir Gráficas en Pantalla

In [ ]:
#GRÁFICA EN CARTESIANAS

#Tamaño de la impresión
plt.figure(figsize=(7, 8))

#Formas de mostrar los datos
plt.scatter(corriente, voltaje, color='blue',label = 'Curva1')

plt.vlines(valor,desde,hasta,lw=0.8,color='orange')

plt.plot(corriente, voltaje, color='blue', marker='o', label = 'Curva1')
plt.plot(dff['Suavizado'], label=f'Suavizado (ventana={window_size})', linestyle='--', linewidth=2)

#Para gráficos con barras de error se necesitan calcular las incertidumbres:
Ecorriente = []; Evoltaje = []
plt.errorbar(corriente, voltaje, errorX=Ecorriente, errorY=Evoltaje, fmt='o', color='red', label='Error $C_1$')

#Límites de ejes
#plt.ylim(0, 450)
#plt.xlim(0, np.pi)

#Nunca poner título de gráficas en artículos, para eso está el caption
#plt.title('Gráfico de Corriente vs. Voltaje')

#Etiquetas de ejes
plt.xlabel(r'$U_2$ [$V$]',fontsize=13)
plt.ylabel(r'$I$ [$nA$]',fontsize=13)

#Para mostrar la gráficas y sus cositas
plt.legend(fontsize=12, loc = 'upper right')
plt.grid(True, alpha=0.3)
plt.savefig('FH_corriente_vs_voltaje.png',dpi=300) #para guardarla
plt.show()

In [ ]:
#GRÁFICA EN POLARES

plt.figure(figsize=(6, 6))
ax = plt.subplot(111, projection='polar')

# Graficar los puntos en coordenadas polares
plt.scatter(theta, r, label='Puntos', color='b', marker='o')

# Configurar el aspecto de la gráfica polar
plt.grid(True)
plt.legend()

# Mostrar la gráfica
plt.show()


## Otros :v

In [ ]:
# Coordenadas de los puntos
x = [[0, 0], [0, 0], [3.8, 6.8], [-3.7, -7.3], [2.2, 3.5], [-1.8, -3.4], [-2.1, -3.6], [2.0, 3.4]]
y = [[-4, -7], [4.1, 7.1], [0, 0], [0, 0], [1.8, 3.5], [2, 3.5], [-2.2, -3.5], [-1.8, -3.3]]

# Función para calcular la distancia euclidiana entre dos puntos
def distancia_euclidiana(x, y,i):
    return np.sqrt((x[i][1] - x[i][0])**2 + (y[i][1] - y[i][0])**2)

# Calcular las distancias entre cada par de puntos
distancias = []

for j in range(8):
    distancia = distancia_euclidiana(x,y,j)
    distancias.append(distancia)